# Build a Mixtral Chatbot with Panel

[Build a Mixtral Chatbot with Panel](https://blog.holoviz.org/posts/mixtral/) by Andrew Huang, Philipp Rudiger, Sophia Yang (December 13, 2023)

[Model Card for Mixtral-8x7B](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1)

## Run Mixtral with Mistral API

[Generate a Mistral AI API key](https://console.mistral.ai/users/api-keys/)

In [1]:
# Install mistralai and panel in the Python environment
# !pip3 install mistralai panel

- No streaming:

In [2]:
# Import modules
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [3]:
# Load Mistral AI API key
api_key=os.environ['MISTRAL_API_KEY']

In [4]:
# Set client with API key
client = MistralClient(api_key=api_key)

In [5]:
# Set chat endpoint with the Mistral API
model = 'mistral-tiny'

In [6]:
# Messages
messages = [
    ChatMessage(
        role='user', 
        content='What is the best French cheese?'
    )
]

In [7]:
# No streaming 
chat_response = client.chat(
    model=model,
    messages=messages,
)

In [8]:
# Display chat response
chat_response

ChatCompletionResponse(id='2481801e6ded4957b195871de497ef40', object='chat.completion', created=1705338157, model='mistral-tiny', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='Determining the "best" French cheese is subjective and depends on personal preferences, as different cheeses offer unique flavors, textures, and milk sources. Here are some popular French cheeses that are renowned for their qualities:\n\n1. Roquefort: A blue-veined cheese from the Massif Central region, made from sheep\'s milk. It is known for its pungent aroma and tangy, salty flavor.\n\n2. Comté: A hard, nutty cheese from the Franche-Comté region, made from cow\'s milk. It has a rich, complex flavor with notes of caramel and butter.\n\n3. Camembert de Normandie: A soft, creamy, and pungent cheese from the Normandy region, made from cow\'s milk. It has a distinctive mushroomy and earthy flavor.\n\n4. Brie de Meaux: A soft, creamy, and smooth cheese from the Île-de-

In [9]:
# Print chat response
print(chat_response.choices[0].message.content)

Determining the "best" French cheese is subjective and depends on personal preferences, as different cheeses offer unique flavors, textures, and milk sources. Here are some popular French cheeses that are renowned for their qualities:

1. Roquefort: A blue-veined cheese from the Massif Central region, made from sheep's milk. It is known for its pungent aroma and tangy, salty flavor.

2. Comté: A hard, nutty cheese from the Franche-Comté region, made from cow's milk. It has a rich, complex flavor with notes of caramel and butter.

3. Camembert de Normandie: A soft, creamy, and pungent cheese from the Normandy region, made from cow's milk. It has a distinctive mushroomy and earthy flavor.

4. Brie de Meaux: A soft, creamy, and smooth cheese from the Île-de-France region, made from cow's milk. It has a mild, buttery, and slightly sweet flavor with a bloomy rind.

5. Munster: A soft, pungent, and runny cheese from the Alsace region, made from cow's milk. It has a strong, garlicky, and slig

- With streaming:

In [10]:
# Set chat endpoint with the Mistral API
model = 'mistral-small'

In [11]:
# Messages
messages = [
    ChatMessage(
        role='user', 
        content='What is the best French cheese?'
    )
]

In [12]:
# Response with streaming
response = client.chat_stream(
    model=model, 
    messages=messages
)

In [13]:
message = ""
for chunk in response:
    part = chunk.choices[0].delta.content
    if part is not None:
        message += part

In [14]:
# Print message
print(message)

I don't have personal experiences or opinions. However, I can share that France is famous for its diverse and high-quality cheeses, making it challenging to declare a single "best" one. The title of "best" can depend on personal taste preferences.

Some of the most renowned French cheeses include:

1. Brie de Meaux: A soft, creamy, and mild cheese from the Brie region.
2. Comté: A hard cheese from the Franche-Comté region, known for its nutty and slightly sweet flavor.
3. Roquefort: A blue cheese from the Roquefort-sur-Soulzon, characterized by its tangy and strong flavor.
4. Camembert de Normandie: A soft and strong-smelling cheese from Normandy.
5. Chevre (goat cheese): Fresh or aged, goat cheese is popular in France, with varieties such as Chabichou du Poitou and Picodon.

These are just a few examples, and there are many more delicious French cheeses to explore. I recommend trying different types and determining which one suits your taste buds the best.


### Build a Panel chatbot

In [15]:
"""
Demonstrates how to use the `ChatInterface` to create a chatbot using
Mistral API.
"""
# Import libraries
import os
import panel as pn
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

# Initialize Panel extension
pn.extension()

# Define asynchronous callback function
async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):

    # Chatbot model
    model = 'mistral-small'
    
    # Use API to send user's message to model
    messages = [ChatMessage(role='user', content=contents)]
    
    # Process response
    response = client.chat_stream(model=model, messages=messages)

    # Concatenate chunks of response to form complete message
    message = ''
    for chunk in response:
        part = chunk.choices[0].delta.content
        if part is not None:
            message += part
            yield message

# Initialize Mistral client using API key retrieved from environmental variable
client = MistralClient(api_key=os.environ['MISTRAL_API_KEY'])

# Initialize chat interface
chat_interface = pn.chat.ChatInterface(callback=callback, callback_user='Mixtral')

# Initial message is sent to the chat interface
chat_interface.send(
    'Send a message to get a reply from Mixtral!', user='System', respond=False
)

# Deploy chat interface as a server
chat_interface.servable()

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(icon='send...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'TextInput': TextInput(cs...}, callback=<function callback a..., callback_user='Mixtral', show_button_name=True, sizing_mode='stretch_width', widgets=[TextInput(css_classes=['c...])
    [0] ChatMessage(str, avatar='⚙️', reaction_icons=ChatReactionIcons, timestamp=datetime.datetime(2024, ..., user='System')

To interact with this app:

1. In CLI, change directory to folder containing this notebook
2. Run `panel serve mixtral-chatbot.ipynb`
3. Open a new browser tab and type in `http://localhost:5006/mixtral-chatbot` (where Bokeh app is running at)